### Evaluación de modelos de lenguaje

La evaluación de un modelo de lenguaje busca determinar la calidad de sus predicciones y su capacidad de asignar probabilidades a secuencias de palabras de manera realista o útil para una aplicación específica. Existen dos métodos principales para esta evaluación:

1. **Evaluación extrínseca**: Mide cómo el modelo de lenguaje contribuye al desempeño de un sistema completo o de una tarea de procesamiento del lenguaje natural (NLP). Un ejemplo es la evaluación de un modelo en un sistema de reconocimiento de voz para ver si reduce la tasa de error de palabra (WER).  
2. **Evaluación intrínseca**: Examina la capacidad de generalización y ajuste del modelo sin ligarlo directamente a una aplicación más amplia. Dentro de esta categoría, la métrica de *perplejidad* es la más representativa, pues refleja qué tan bien el modelo asigna probabilidades a secuencias de palabras.

En la construcción y evaluación de estos modelos, se sigue un esquema de separación de datos para entrenamiento, ajuste y validación:

- **Conjunto de entrenamiento**: Se usa para aprender los parámetros del modelo, es decir, para estimar las probabilidades de ocurrencia de secuencias de palabras (en un modelo de n-gramas, por ejemplo, se cuentan las apariciones y se obtienen proporciones).  
- **Conjunto de desarrollo (o validación)**: Permite ajustar hiperparámetros (por ejemplo, el valor de *n* en un n-grama, parámetros de regularización o de "suavizado") sin sesgar los resultados.  
- **Conjunto de prueba**: Se emplea únicamente para medir la capacidad real de generalización. No se debe usar en ningún momento durante el entrenamiento o la selección de parámetros, para evitar el sobreajuste y disponer de una métrica confiable.

Mantener una clara separación de estos conjuntos ayuda a que la evaluación sea representativa de cómo funcionará el modelo en datos nuevos.

##### **Conjuntos de entrenamiento, prueba y desarrollo**

Para evitar sesgos y estimar adecuadamente el rendimiento, se necesitan tres partes de datos:

1. **Entrenamiento**: Contiene la mayor cantidad de datos posible, pues el objetivo es que el modelo capte la mayor diversidad de patrones.  
2. **Desarrollo (o validación)**: Sirve para ajustar hiperparámetros y tomar decisiones de arquitectura sin afectar la métrica final que se reportará. Por ejemplo, se puede usar este conjunto para decidir si se implementa un modelo de unigramas, bigramas, trigramas o un modelo neuronal recurrente.  
3. **Prueba**: Se deja completamente aislado durante el proceso. Solo se accede a él al final, cuando el modelo está entrenado y ajustado. Proporciona la métrica final de rendimiento, ofreciendo una estimación de la capacidad de generalización en datos no vistos.

Un punto crucial es que el conjunto de prueba nunca debe influir en el entrenamiento ni en la selección de parámetros. De lo contrario, las mediciones de rendimiento no serían realistas, ya que se incurriría en *overfitting* al conjunto de prueba.

#### Perplejidad como métrica intrínseca de evaluación

La **perplejidad** (también conocida como *PPL*, del inglés *perplexity*) es la medida más común para la evaluación intrínseca de un modelo de lenguaje. Se define como la inversa de la probabilidad (según el modelo) de una secuencia de prueba, elevada a la potencia de 1 sobre el número de palabras de la secuencia.

Para un conjunto de prueba $ W = w_1 w_2 \dots w_N $, la perplejidad se expresa como:

$$
\text{perplexity}(W) 
= P(w_1 w_2 \dots w_N)^{-\frac{1}{N}}
= \sqrt[N]{\frac{1}{P(w_1 w_2 \dots w_N)}}.
$$

Si se usa la regla de la cadena de probabilidades, queda:

$$
\text{perplexity}(W) = \sqrt[N]{\prod_{i=1}^{N} \frac{1}{P(w_i \mid w_1 \dots w_{i-1})}}.
$$

El factor $\frac{1}{N}$ en el exponente normaliza la probabilidad inversa entre las $N$ palabras. Mientras más baja sea la perplejidad en el conjunto de prueba, mejor se considera el modelo, puesto que corresponde a asignar una probabilidad más alta a la secuencia real. Minimizar la perplejidad equivale a maximizar la probabilidad del conjunto de prueba bajo el modelo.

El uso de la probabilidad inversa proviene de la relación con la entropía cruzada en teoría de la información: la perplejidad puede verse como una exponencial de esa entropía, indicando qué tan "sorprendido" está el modelo ante las secuencias reales.


#### Ejemplos de cálculo de perplejidad con diferentes n-gramas

Cuando se calcula la perplejidad con un **modelo de unigramas**, solo se tienen en cuenta frecuencias de palabras individuales. La fórmula para el conjunto de prueba $ W = w_1 \dots w_N $ es:

$$
\text{perplexity}(W) = \sqrt[N]{\prod_{i=1}^{N} \frac{1}{P(w_i)}}.
$$

En un **modelo de bigramas**, la perplejidad se basa en la probabilidad condicional de cada palabra dada la anterior:

$$
\text{perplexity}(W) = \sqrt[N]{\prod_{i=1}^{N} \frac{1}{P(w_i \mid w_{i-1})}}.
$$

Para **trigramas**, se consideran pares de palabras anteriores:

$$
\text{perplexity}(W) = \sqrt[N]{\prod_{i=1}^{N} \frac{1}{P(w_i \mid w_{i-2}, w_{i-1})}}.
$$

Algunos modelos añaden tokens especiales de inicio `<s>` y fin de oración `</s>`, contabilizando dichos símbolos según convenga. Normalmente, el token de fin de oración sí se cuenta para la perplejidad, mientras que el token de inicio se trata con cuidado para no distorsionar la distribución.

#### **Ejemplos prácticos de cálculo de perplejidad** 

A continuación se muestra una serie de bloques de código en Python que ilustran cómo calcular la perplejidad de un modelo de unigramas, bigramas y trigramas. Se parte de un texto de ejemplo, el cual se tokeniza, y luego se cuentan las frecuencias para estimar probabilidades. Finalmente, se evalúa un conjunto de prueba simulado.

In [ ]:
import math
import nltk
from nltk.util import ngrams
from collections import Counter, defaultdict
nltk.download('punkt')

# Simulamos un conjunto de datos de ejemplo con 38 millones de palabras (simplificado aquí)
texto = """<s> This is the first sentence . </s> <s> This is the second sentence . </s> <s> This is the third sentence . </s>"""

# Tokenizamos el texto en palabras y mantenemos <s> y </s> como tokens completos
tokens = [token for token in nltk.word_tokenize(texto) if token not in ['<', '>']]
tokens = ['<s>' if token == 's' else token for token in tokens]
tokens = ['</s>' if token == '/s' else token for token in tokens]

# Función para calcular la perplejidad del modelo de unigrama
def unigram_perplexity(test_tokens, unigram_probs):
    N = len(test_tokens)
    perplexity = 1
    for token in test_tokens:
        prob = unigram_probs.get(token, 1e-10)  # Evitar probabilidad cero
        perplexity *= 1/prob
    return perplexity ** (1/N)

# Función para calcular la perplejidad del modelo de bigrama
def bigram_perplexity(test_tokens, bigram_probs):
    N = len(test_tokens) - 1  # número de bigramas
    perplexity = 1
    bigrams = list(ngrams(test_tokens, 2))
    for bigram in bigrams:
        prob = bigram_probs.get(bigram, 1e-10)  # Evitar probabilidad cero
        perplexity *= 1/prob
    return perplexity ** (1/N)

# Función para calcular la perplejidad del modelo de trigramas
def trigram_perplexity(test_tokens, trigram_probs):
    N = len(test_tokens) - 2  # número de trigramas
    perplexity = 1
    trigrams = list(ngrams(test_tokens, 3))
    for trigram in trigrams:
        prob = trigram_probs.get(trigram, 1e-10)  # Evitar probabilidad cero
        perplexity *= 1/prob
    return perplexity ** (1/N)

# Creamos un modelo de unigramas, bigramas y trigramas basado en frecuencias
unigrams = Counter(tokens)
bigrams = Counter(ngrams(tokens, 2))
trigrams = Counter(ngrams(tokens, 3))

# Calculamos probabilidades de unigramas
total_unigrams = sum(unigrams.values())
unigram_probs = {word: count/total_unigrams for word, count in unigrams.items()}

# Calculamos probabilidades de bigramas
bigram_probs = {}
for bigram, count in bigrams.items():
    first_word = bigram[0]
    bigram_probs[bigram] = count / unigrams[first_word]

# Calculamos probabilidades de trigramas
trigram_probs = {}
for trigram, count in trigrams.items():
    first_two_words = (trigram[0], trigram[1])
    trigram_probs[trigram] = count / bigrams[first_two_words]

# Simulamos un conjunto de prueba de 1.5 millones de palabras (simplificado)
test_text = """<s> This is the first test sentence . </s> <s> This is another sentence . </s>"""
test_tokens = [token for token in nltk.word_tokenize(test_text) if token not in ['<', '>']]
test_tokens = ['<s>' if token == 's' else token for token in test_tokens]
test_tokens = ['</s>' if token == '/s' else token for token in test_tokens]

# Calculamos la perplejidad para unigramas, bigramas y trigramas
unigram_ppl = unigram_perplexity(test_tokens, unigram_probs)
bigram_ppl = bigram_perplexity(test_tokens, bigram_probs)
trigram_ppl = trigram_perplexity(test_tokens, trigram_probs)

print(f"Perplejidad del modelo de unigramas: {unigram_ppl}")
print(f"Perplejidad del modelo de bigramas: {bigram_ppl}")
print(f"Perplejidad del modelo de trigramas: {trigram_ppl}")


#### Perplejidad como factor de ramificación promedio ponderado 

La perplejidad también puede interpretarse como el **factor de ramificación promedio ponderado**. El factor de ramificación simple de un idioma se refiere al número de palabras que pueden aparecer después de un contexto dado; si todo es igualmente probable, ese número es constante. Sin embargo, en la práctica, las probabilidades no son uniformes. Por ende, la perplejidad integra la variación en la probabilidad de cada palabra, convirtiéndose en un factor de ramificación "ponderado".

Por ejemplo, si se tiene un lenguaje artificial donde cualquier dígito (0-9) puede aparecer con la misma probabilidad $1/10$, la perplejidad es 10, lo cual coincide con el número de opciones posibles a cada paso. Pero si uno de los dígitos, por ejemplo el 0, ocurre mucho más que los demás (digamos 90% de las veces), la perplejidad resultante es menor que 10, porque en la práctica hay menos opciones plausibles la mayor parte del tiempo.

Este enfoque ayuda a entender por qué la perplejidad disminuye cuando hay un fuerte sesgo probabilístico en la próxima palabra; el modelo se "confunde" menos sobre qué palabra escoger, quedando un factor de ramificación efectivo menor.


In [ ]:
import math

# Vocabulario de 10 dígitos (0-9)
vocab_size = 10
# Probabilidad uniforme de cada dígito
prob_digit = 1 / vocab_size
# Simulamos una secuencia de prueba
test_sequence = "0030000300"  # Ejemplo de secuencia de prueba
# Calculamos el número de dígitos en la secuencia de prueba
N = len(test_sequence)

# La probabilidad de la secuencia completa es el producto de la probabilidad de cada dígito
prob_sequence = prob_digit ** N

# Calculamos la perplejidad basada en la probabilidad de la secuencia
def calculate_perplexity(N, prob_sequence):
    return (1 / prob_sequence) ** (1 / N)
# Cálculo de perplejidad
perplexity = calculate_perplexity(N, prob_sequence)

print(f"Perplejidad de la secuencia '{test_sequence}': {perplexity:.2f}")


Aquí, si cada dígito se asume con igual probabilidad, la perplejidad es 10. En cambio, si el dígito '0' aparece con probabilidad 0.9 y los demás con 0.01, la perplejidad real para una secuencia mayormente compuesta de '0' disminuirá.


#### Muestreo y visualización de modelos de lenguaje

Otro aspecto importante es **muestrear** oraciones a partir de un modelo de lenguaje para comprender la clase de secuencias que produce. En un modelo de unigramas, se eligen palabras según su frecuencia global, mientras que en un modelo de bigramas o trigramas, la elección de la próxima palabra depende de la palabra o palabras previas.

##### **Muestreo de unigramas**

Se puede asignar a cada palabra un intervalo dentro de `[0,1]` cuya amplitud depende de su probabilidad. Luego, un número aleatorio en `[0,1]` indica cuál palabra se elige. Se repite el proceso hasta que aparece el token `</s>`.

##### **Muestreo de bigramas**

Se extiende la idea: se empieza con `<s>` y se selecciona una palabra según la distribución de probabilidades condicionales que comienzan con `<s>`. Si se elige la palabra `the`, entonces para la siguiente elección se acota a los bigramas que comienzan con `the`, y así sucesivamente hasta llegar a `</s>`.

Esta técnica la sugirieron Shannon (1948) y Miller y Selfridge (1950) para visualizar las propiedades del modelo. Permite ver cuán "coherente" puede sonar el texto generado, y cómo las probabilidades reflejan las regularidades de un corpus.



#### **Ejemplo de generación de palabras aleatorias**

1. **Distribución de palabras en el intervalo [0, 1]:**
   Supongamos que tenemos un vocabulario muy simplificado con las siguientes palabras: `['the', 'dog', 'barks', 'in', 'the', 'park', '</s>']`. Aquí también incluimos el token de final de oración `</s>`. La probabilidad de cada palabra depende de su frecuencia en el idioma.

   Podemos asignar intervalos en la línea de probabilidad `[0, 1]` de la siguiente manera (usando probabilidades simplificadas para el ejemplo):

   - `the`: Probabilidad 0.4 --> Intervalo `[0.0, 0.4)`
   - `dog`: Probabilidad 0.2 --> Intervalo `[0.4, 0.6)`
   - `barks`: Probabilidad 0.1 --> Intervalo `[0.6, 0.7)`
   - `in`: Probabilidad 0.1 --> Intervalo `[0.7, 0.8)`
   - `park`: Probabilidad 0.05 --> Intervalo `[0.8, 0.85)`
   - `</s>`: Probabilidad 0.15 --> Intervalo `[0.85, 1.0)`

2. **Generación de una palabra:**
   Supongamos que generamos un número aleatorio entre 0 y 1, y el número es `0.32`. Este número cae en el intervalo de la palabra `the`, por lo que la primera palabra seleccionada es **the**.

3. **Continuar generando palabras:**
   Generamos otro número aleatorio, digamos `0.72`. Este número cae en el intervalo de la palabra `in`, por lo que la siguiente palabra seleccionada es **in**.

4. **Finalizar la oración:**
   Continuamos generando números aleatorios y seleccionando palabras hasta que finalmente generamos un número en el intervalo de `</s>`. Esto indica que hemos llegado al final de la oración.

**Extensión a bigramas**

Ahora, extendamos este concepto para trabajar con **bigramas**.

1. **Distribución de bigramas:**
   En lugar de asignar probabilidades a palabras individuales, asignamos probabilidades a secuencias de dos palabras (bigramas). Por ejemplo, supongamos que tenemos las siguientes probabilidades de bigramas basadas en un corpus de texto:

   - `<s> the`: 0.5 --> Intervalo `[0.0, 0.5)`
   - `<s> dog`: 0.3 --> Intervalo `[0.5, 0.8)`
   - `<s> park`: 0.2 --> Intervalo `[0.8, 1.0)`

   Así, la primera palabra de la oración es elegida según el bigrama que empieza con `<s>`. Generamos un número aleatorio, digamos `0.4`. Este número cae en el intervalo de `<s> the`, por lo que generamos el bigrama **the** como la primera palabra.

2. **Continuar con bigramas:**
   Ahora, queremos generar un bigrama que comience con la palabra `the`. Supongamos que las probabilidades para los bigramas que comienzan con `the` son:

   - `the dog`: 0.4 --> Intervalo `[0.0, 0.4)`
   - `the park`: 0.6 --> Intervalo `[0.4, 1.0)`

   Generamos otro número aleatorio, digamos `0.3`. Este número cae en el intervalo de `the dog`, por lo que el siguiente bigrama es **dog**. Ahora tenemos la secuencia `"<s> the dog"`.

3. **Continuar hasta finalizar:**
   Supongamos que las probabilidades de los bigramas que comienzan con `dog` son:

   - `dog barks`: 0.7 --> Intervalo `[0.0, 0.7)`
   - `dog </s>`: 0.3 --> Intervalo `[0.7, 1.0)`

   Generamos otro número aleatorio, digamos `0.9`, lo que nos lleva al bigrama `dog </s>`. Esto indica el final de la oración.


Usando este proceso, generamos la oración:

```
<s> the dog </s>
```

El proceso de selección aleatoria a lo largo de la línea de probabilidad para palabras y bigramas nos permite generar oraciones de manera coherente, basadas en las probabilidades observadas en un corpus de texto. Cada nueva palabra o secuencia de palabras (bigrama) es seleccionada en función de las frecuencias relativas de las secuencias precedentes.

El siguiente bloque de código en Python muestra cómo generar palabras según la probabilidad de unigramas, y cómo visualizar la distribución acumulada:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
from collections import Counter

# Función para calcular las probabilidades de unigramas a partir de un texto
def calculate_unigram_probs(texto):
    words = texto.split()
    total_words = len(words)
    word_counts = Counter(words)
    vocab = list(word_counts.keys())
    probs = [word_counts[word] / total_words for word in vocab]
    cumulative_probs = np.cumsum(probs)
    return vocab, probs, cumulative_probs

# Función para generar una palabra basada en un número aleatorio entre 0 y 1
def generate_word(vocab, cumulative_probs):
    rand_val = random.random()  # Elegimos un valor aleatorio entre 0 y 1
    for i, cum_prob in enumerate(cumulative_probs):
        if rand_val <= cum_prob:
            return vocab[i]

# Función para generar una oración completa
def generate_sentence(vocab, cumulative_probs):
    sentence = []
    while True:
        word = generate_word(vocab, cumulative_probs)
        if word == '</s>':  # Si obtenemos el token de fin de oración, terminamos
            break
        sentence.append(word)
    return ' '.join(sentence)

# Función para visualizar la distribución de unigramas
def visualize_unigram_sampling(vocab, probs, cumulative_probs):
    fig, ax = plt.subplots(figsize=(12, 3))  # Ajustamos el tamaño del gráfico
    
    # Graficamos las barras que representan los intervalos de probabilidad
    for i, word in enumerate(vocab):
        ax.barh(0, cumulative_probs[i], color='blue', alpha=0.5, edgecolor='black', 
                left=(cumulative_probs[i-1] if i > 0 else 0), height=0.5)
    
    # Mostrar el número de probabilidad acumulada en el eje X
    ax.set_xticks(np.round(cumulative_probs, 2))
    ax.set_xticklabels(np.round(cumulative_probs, 2), fontsize=10)
    ax.set_yticks([])
    
    # Etiquetar las palabras encima de las barras
    for i, word in enumerate(vocab):
        mid_point = (cumulative_probs[i-1] if i > 0 else 0) + (probs[i] / 2)
        plt.text(mid_point, 0.1, word, fontsize=12, va='center', ha='center', color='black')
    
    ax.set_xlim(0, 1)
    plt.title('Visualización de la distribución de muestreo de unigramas', fontsize=14)
    plt.show()

# Texto de ejemplo para generar el modelo de unigramas
text = "the of a to in the the of a in polyphonic however to in </s>"

# Calculamos las probabilidades de unigramas y las probabilidades acumuladas
vocab, probs, cumulative_probs = calculate_unigram_probs(texto)

# Generar una oración de ejemplo
sentence = generate_sentence(vocab, cumulative_probs)
print("Oración generada:", sentence)

# Visualizar la distribución de probabilidades acumuladas
visualize_unigram_sampling(vocab, probs, cumulative_probs)


Este tipo de muestreo aleatorio puro puede producir resultados poco coherentes en algunos casos (modelo de unigramas) o más coherentes cuando se emplean bigramas y trigramas, pues se restringe la siguiente palabra a las que tengan mayor probabilidad condicional dada la palabra previa.

#### **Técnicas de muestreo**  

Al generar texto con modelos de lenguaje, existen diversas estrategias de muestreo para seleccionar la siguiente palabra. A continuación, exploramos tres enfoques principales.  

##### **1. Muestreo aleatorio puro**  

En este método, la siguiente palabra se selecciona **aleatoriamente** según la distribución de probabilidad del modelo. Es similar a lanzar un dado, donde cada cara tiene un peso diferente según la probabilidad de cada palabra.  

Supongamos que el modelo ha calculado las siguientes probabilidades para las próximas palabras:  

- $ P(\text{cat}) = 0.05 $  
- $ P(\text{is}) = 0.04 $  
- $ P(\text{jumping}) = 0.03 $  
- $ P(\text{on}) = 0.02 $  
- $ P(\text{the}) = 0.01 $  

El modelo genera un número aleatorio entre 0 y 1. Supongamos que el número aleatorio generado es **0.045**.  

Las probabilidades acumuladas serían:  
- "cat": $ 0.05 $  
- "is": $ 0.05 + 0.04 = 0.09 $  
- "jumping": $ 0.09 + 0.03 = 0.12 $  
- ...  

Dado que **0.045** está entre **0.05 (cat) y 0.09 (is)**, el modelo seleccionaría **"is"**.  

Este enfoque introduce **variabilidad** en el texto, pero también puede generar oraciones poco coherentes, como:  

> **"The is jumping."**  

##### **2. Muestreo Top-K**  

El **muestreo top-k** selecciona solo las **k palabras más probables**, descartando las demás y renormalizando la distribución de probabilidad entre las palabras seleccionadas.  

Supongamos que **$ k = 4 $**. Las palabras más probables son:  

- **"cat"** $ P(\text{cat}) = 0.05 $  
- **"is"** $ P(\text{is}) = 0.04 $  
- **"jumping"** $ P(\text{jumping}) = 0.03 $  
- **"on"** $ P(\text{on}) = 0.02 $  

Las demás palabras son **descartadas**. Luego, renormalizamos las probabilidades entre estas cuatro palabras:  

$$
P'(\text{cat}) = \frac{0.05}{0.05 + 0.04 + 0.03 + 0.02} = 0.31
$$

$$
P'(\text{is}) = \frac{0.04}{0.14} = 0.25
$$

$$
P'(\text{jumping}) = \frac{0.03}{0.14} = 0.19
$$

$$
P'(\text{on}) = \frac{0.02}{0.14} = 0.12
$$

Luego, el modelo selecciona una palabra de esta distribución renormalizada.  

Este método **genera frases más coherentes**, como:  

> **"The cat is jumping."**  

Sin embargo, **reduce la diversidad** del texto al limitar las opciones disponibles.  


##### **3. Muestreo Top-P (núcleo de muestreo)**  

En el **muestreo top-p**, en lugar de elegir un número fijo de palabras ($ k $), seleccionamos **un subconjunto de palabras cuya suma acumulada de probabilidades alcanza al menos $ p $**.  

Supongamos que establecemos $ p = 0.7 $ y el modelo ha calculado las siguientes probabilidades:  

- **"cat"** $ P(\text{cat}) = 0.30 $  
- **"is"** $ P(\text{is}) = 0.25 $  
- **"jumping"** $ P(\text{jumping}) = 0.10 $  
- **"on"** $ P(\text{on}) = 0.05 $  
- **"the"** $ P(\text{the}) = 0.03 $  
- **"roof"** $ P(\text{roof}) = 0.02 $  
- **"under"** $ P(\text{under}) = 0.02 $  
- ...  

El modelo seleccionará palabras hasta que la **suma acumulada de probabilidades alcance al menos 0.7**:  

- "cat" → $ 0.30 $  
- "is" → $ 0.30 + 0.25 = 0.55 $  
- "jumping" → $ 0.55 + 0.10 = 0.65 $  
- "on" → $ 0.65 + 0.05 = 0.70 $ 

Las palabras seleccionadas son: **"cat," "is," "jumping" y "on"**.  

Luego, la probabilidad se renormaliza y el modelo selecciona una palabra dentro de este subconjunto.  

Este enfoque mantiene un **equilibrio entre diversidad y coherencia**. Puede generar oraciones como:  

> **"The cat is jumping on the roof."**  

Si **$ p $** es muy bajo, la diversidad se reduce demasiado; si **$ p $** es muy alto, aumenta la variabilidad y el texto puede volverse incoherente.  

##### **Comparación de las técnicas**  

| Método         | Características | Ventajas | Desventajas |
|---------------|----------------|----------|-------------|
| **Muestreo aleatorio puro** | Selecciona palabras de acuerdo a la distribución del modelo. | Alta diversidad. | Puede generar frases incoherentes. |
| **Muestreo top-k** | Mantiene solo las $ k $ palabras más probables. | Mayor coherencia, menos ruido. | Reduce la diversidad del texto. |
| **Muestreo top-p** | Selecciona un subconjunto hasta alcanzar un umbral de probabilidad $ p $. | Equilibra coherencia y diversidad. | Sensible al valor de $ p $, puede generar resultados inesperados. |



#### Generalización, *sparsity* y vocabularios

Los modelos de n-gramas pueden enfrentar el problema de la falta de datos o *sparsity*: muchas combinaciones de palabras no aparecen en el corpus de entrenamiento, resultando en probabilidades condicionales de cero para secuencias que podrían ser perfectamente válidas en el idioma.

En un corpus como el *Wall Street Journal*, el bigrama `denied the` puede ser seguido por `allegations`, `speculation`, `rumors`, `report`. Si en el conjunto de prueba aparece `offer` o `loan` después de `denied the`, el modelo de n-gramas puro asigna probabilidad 0, lo cual causa problemas al calcular la perplejidad (no se puede dividir entre 0). Para solucionar esto, se usan técnicas de **suavizado** como Laplace (add-one), Good-Turing, Kneser-Ney, etc., que redistribuyen la probabilidad dando masa no nula a eventos no vistos.

Otro reto surge cuando aparecen palabras nuevas que no están en el vocabulario del modelo (OOV). Los modelos de vocabulario cerrado asumen que todas las palabras del conjunto de prueba se han visto en el entrenamiento. Una solución alternativa es convertir cualquier palabra fuera de un vocabulario básico en `<UNK>` dentro del entrenamiento, estimando así la probabilidad de `<UNK>` como una palabra más. Sin embargo, eso distorsiona la perplejidad si se comparan modelos con distintos vocabularios.

En la actualidad, los modelos neuronales suelen usar *tokenización de subpalabras* (p. ej., Byte-Pair Encoding, WordPiece o SentencePiece), evitando el problema de las palabras desconocidas, porque una palabra nunca vista se descompone en subunidades que sí están en el vocabulario. Así, la perplejidad se calcula sobre secuencias de subpalabras, y no se depende de un vocabulario cerrado.


#### **Oraciones generadas en diferentes dominios**

Para ilustrar la importancia de entrenar en el género apropiado, pueden compararse oraciones generadas por un modelo de n-gramas entrenado en las obras de Shakespeare y otro entrenado en noticias financieras del *Wall Street Journal*. Aunque ambos estén en inglés, las secuencias generadas difieren ampliamente, mostrando que el lenguaje es sensible al dominio, vocabulario y estilo:

- **Pseudo-Shakespeare (4-gramas)**:  
  *“King Henry. What! I will go seek the traitor Gloucester. Exeunt some of the watch. A great banquet serv’d in; It cannot be but so.”*  

- **Pseudo-WSJ (3-gramas)**:  
  *“They also point to ninety nine point six billion dollars from two hundred four oh six three percent of the rates of interest stores as Mexico and Brazil on market conditions.”*  

Estos modelos tienen propósitos distintos y difícilmente serían intercambiables.


#### **Ejercicios**

**Ejercicio 1: Modificar el modelo de unigramas para manejar palabras desconocidas**

En este ejercicio, vas a agregar una funcionalidad para manejar palabras desconocidas (OOV). Usa un vocabulario cerrado que solo incluya palabras vistas en el conjunto de entrenamiento, y reemplaza las palabras desconocidas en el conjunto de prueba con un token especial `<UNK>`.

1. Modifica el código para generar una oración utilizando un vocabulario cerrado. Si una palabra desconocida aparece en el corpus de prueba, debe ser reemplazada por `<UNK>`.
2. Prueba con un corpus de prueba que contenga una palabra fuera del vocabulario, por ejemplo, `extraterrestrial`.

**Pista:** Puedes utilizar la función `convert_to_unk` del ejemplo anterior.


**Ejercicio 2: Implementar un modelo de bigrama**

En este ejercicio, vas a extender el código actual para trabajar con **bigramas**. Un modelo de bigramas elige la siguiente palabra en función de la palabra anterior.

1. Modifica el código para implementar un modelo de bigramas.
2. Usa el corpus de entrenamiento para calcular las probabilidades de los bigramas.
3. Genera una oración utilizando el modelo de bigramas, comenzando con un token especial `<s>`.
4. Visualiza la distribución de las probabilidades de los bigramas de manera similar a cómo se visualizan los unigramas.

**Pistas:**
- Usa `zip` para generar los bigramas del corpus.
- Asegúrate de incluir un token de inicio `<s>` en cada oración para iniciar el muestreo del bigrama.


**Ejercicio 3: Muestreo con suavizado**

En los modelos de n-gramas, cuando ciertas combinaciones de palabras no aparecen en el conjunto de entrenamiento, tienen una probabilidad de cero. En este ejercicio, vas a implementar **suavizado de Laplace** (Add-One Smoothing) para evitar probabilidades de cero.

1. Implementa el suavizado de Laplace en el modelo de unigramas.
2. Modifica el código para que todas las palabras, incluso las que no están en el corpus, tengan una pequeña probabilidad mayor a cero.
3. Genera una oración con el modelo suavizado.

**Pista:** El suavizado de Laplace implica sumar 1 a la frecuencia de cada palabra y ajustar la suma total de palabras en consecuencia.


**Ejercicio 4: Visualización comparativa de unigramas y bigrama**

Este ejercicio consiste en comparar las distribuciones generadas por un modelo de unigramas y un modelo de bigramas.

1. Modifica el código para que genere dos gráficos: uno para la visualización de las probabilidades acumuladas de unigramas y otro para bigramas.
2. Compara cómo cambia la distribución entre los dos modelos.
3. Discute cómo los modelos de unigramas y bigramas afectan la coherencia de las oraciones generadas.

**Pista:**
- Al generar bigramas, debes calcular las probabilidades condicionales de una palabra dado la anterior.
- Usa `Counter` para contar las ocurrencias de bigramas en el corpus.


**Ejercicio 5: Generalización y sparsity**

El **problema de sparsity** ocurre cuando un modelo de n-gramas no ha visto ciertos n-gramas en el conjunto de entrenamiento. En este ejercicio, vas a explorar este problema con un modelo de bigramas.

1. Entrena un modelo de bigramas en un corpus pequeño.
2. Intenta generar oraciones con el modelo. ¿Qué sucede cuando intentas generar una oración con una combinación de palabras que no aparece en el conjunto de entrenamiento?
3. Implementa un mecanismo que detecte estas combinaciones no vistas y las maneje adecuadamente (por ejemplo, generando un token `<UNK>` para secuencias no vistas).



**Ejercicio 6: Muestreo condicional basado en frecuencias**

1. Usa un modelo de bigramas para generar oraciones, pero esta vez ajusta la probabilidad de las palabras en función de la palabra anterior.
2. Usa una técnica similar a la de los unigramas, pero ahora al generar cada palabra, elige un bigrama en función de la última palabra generada.

**Pistas:**

- Calcula las probabilidades condicionales $P(w_n | w_{n-1})$.
- Usa el valor aleatorio para seleccionar el bigrama según las probabilidades acumuladas, como se hizo con los unigramas.


**Ejercicio 7: Experimentación con vocabularios cerrados y abiertos**

1. Usa un vocabulario cerrado para entrenar el modelo de unigramas.
2. Luego, expande el vocabulario para incluir más palabras y usa un vocabulario abierto. Compara cómo cambian las oraciones generadas y su coherencia entre ambos enfoques.
3. Discute las ventajas y desventajas de usar vocabularios cerrados frente a vocabularios abiertos.


**Ejercicio 8: Generación de texto con diferentes n-gramas**

1. Modifica el código para que pueda generar oraciones utilizando unigramas, bigramas y trigramas.
2. Compara la coherencia de las oraciones generadas por cada modelo. ¿Qué modelo genera oraciones más coherentes y por qué?
3. ¿Qué sucede cuando entrenas con un modelo de trigramas en un corpus pequeño? Explora el problema del **sparsity**.

**Pista:** El modelo de trigramas debe calcular las probabilidades de la palabra actual dado las dos palabras anteriores.


**Ejercicio 9: Exploración de subpalabras y OOV**

1. Implementa una función que divida palabras en subpalabras o letras individuales cuando no se encuentren en el vocabulario (es decir, maneja palabras OOV utilizando tokenización de subpalabras).
2. Prueba el modelo generando oraciones en las que aparezcan palabras OOV y comprueba si la tokenización de subpalabras ayuda a generar texto más coherente.



**Ejercicio 10: Comparación de probabilidades acumuladas en unigramas**

1. Crea una función que permita seleccionar un subconjunto del vocabulario y mostrar las probabilidades acumuladas solo para ese subconjunto.
2. Genera oraciones utilizando solo palabras del subconjunto y visualiza cómo cambian las probabilidades acumuladas.

**Pista:** Modifica la función `visualize_unigram_sampling` para que acepte un subconjunto de palabras y sus probabilidades.


In [ ]:
## Respuestas